<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li></ul></div>

# Covid 19 Notebook

1) Auto Dump Covid Seq
2) What are the protens in covid
https://www.nytimes.com/interactive/2020/04/03/science/coronavirus-genome-bad-news-wrapped-in-protein.html
3) Organize protein 


<strong>Goals and notes</strong>

In [1]:
%%bash

pwd
ls

## Libraries

In [2]:
# Basic Packages
import numpy as np
import pandas as pd

# Ploting Libraries
import matplotlib.pyplot as plt

# Bio Libraries
from Bio import SeqIO

## Exploratory Data Analysis

In [3]:
# Initialize dictionary
sequence_dict = dict()
protein_set = set()
with open("sequences.fasta", "rU") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence_dict[record.id] = {}
        sequence_dict[record.id]["name"] = record.id
        sequence_dict[record.id]["length"] = len(record.seq)
        sequence_dict[record.id]["sequence"] = record.seq
        
        # Spliting up the pipes in the description     
        sequence_dict[record.id]["description"] = record.description.split("|")[1]
        sequence_dict[record.id]["type"] = record.description.split("|")[3]
        sequence_dict[record.id]["species"] = record.description.split("|")[4]
        sequence_dict[record.id]["country"] = record.description.split("|")[6]
        
        # These sequences are very much out of alignment, but I want to look at protein to 
        # consider phylogenetics this line below builds an amino acid translation from DNA sequences.
        sequence_dict[record.id]["amino_list"] = str(record.seq.translate()).split("*")
        
        # I am curious about the proteins used in each variant. Making set to explore values
        protein_set |= set(sequence_dict[record.id]["amino_list"])        

<ipython-input-3-33c3f6cc12ef>:4: DeprecationWarning: 'U' mode is deprecated
  with open("sequences.fasta", "rU") as fasta_file:
/home/michael/anaconda3/envs/bio/lib/python3.8/site-packages/Bio/Seq.py:2855: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [4]:
# There are a lot of entries here, so we need to figure out a way to categorize them:
protein_list = list(protein_set)

# Make a sorted list of proteins
protein_list.sort(key=lambda item: (-len(item), item))

In [5]:
protein_list

['...',
 'DLSAPVVYVIDVPHAFPLLQTLMPVGIILLDLITSIIRL',
 'DLSAPVVYVIDVPHAIPLLQTLMPVGIILLDLITSIIRL',
 'FPGNKPTNFRSLVDLFSKRTLKSVWLSLGCMLSALTQYN',
 'FPGNKPXNFRSLVDLFSKRTLKSVWLSLGCMLSALTQYN',
 'HSPRVFTLFATCCCCLQQFTHTFCSLLLALKPLFSIFML',
 'IQRRNRPTHASKSPKRNYLLRGRNTSHRSVNRGSCLENW',
 'IQRRNWPTHASKSPKRNYLLRGRNTSHRSVNRGSCLENW',
 'IQRRNWPTHASKSPKRNYLLRGRNTSHRSVNRRSCLENW',
 'IQRRNWPTHASKSPKRSYLLRGRNTSHRSVNRGSCLENW',
 'KSHHIFTEATRSTIECTVNNARESCLYGRALMCKINFSS',
 'LFEXXXXXXXXXXXXXXXXIKCLPRFFNLLNRCFRCFNV',
 'LPGNKPLNFRSLVDLFSKRTLKSVWLSLGCMLSALTQYN',
 'NNLLVKLLKLHWLVHQFVLTGLCCSKSKTQKSTVPLHLI',
 'NNLLVKLLKLHWLVHQFVLTGLFCSKSKTQKSTVPLHLI',
 'RTLSQFFIIKTMFLCLKQNVGLRLTLLKDLLNFALNIQC',
 'RTLSQFFIIKTMFLCLKQNVGLRLTLLKDLMNFALNIQC',
 'SDKELITNIGRKLHNLPPALQRSSECRALAWKSHLRERG',
 'SPFSLSLCFSLLLAEYKLCKNNNEALALLEMPFQKPITL',
 'VQTLVLNMLPSSSTIKLLMSLKNMSKFTQSTFHPELLIQ',
 'VQTLVLNMLPSSSTIKLLMSLKNMSKFTQSTVHPELLIQ',
 'ADTILLNTVFMLSAQMLTKVKTFNFLRVLMKILISTKF',
 'CFFFCTYSVDGYVHTFSTFLDNNCLYHLYFHKAFLLVL',
 'CFFFSTYSVDGYVHTFSTFLDNNC

In [6]:
count = 0
protein_dict = dict()

for index in range(len(protein_list)):
    if protein_list[index][0:5] == protein_list[index - 1][0:5]:
        count += 1
    else:
        count = 0
        
    protein_dict[protein_list[index]] = dict()
    
    size = len(protein_list[index]) 
    if size > 100: 
        print(size, protein_list[index][0:10])
    #if size > 30:
    # Here we are going to make two different classifications, family and variant
    protein_dict[protein_list[index]]["variant"] = "{}-{}-{}".format(protein_list[index][0:7], size, count)
    protein_dict[protein_list[index]]["family"] = "{}-{}".format(protein_list[index][0:7], size)

4447 LITVVDRTRV
4447 LITVVDRTRV
4444 AGDRTRVTRL
4441 RTRVTRLSSA
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESLVP
4409 PKGKMESXVP
4408 KGK

In [7]:
# This next step can be done in pandas, but it is quicker to iterate through the dict
for record in sequence_dict:  
    for entry in protein_set:
            sequence_dict[record][protein_dict[entry]["variant"]] = sequence_dict[record]["amino_list"].count(entry)

In [8]:
df = pd.DataFrame.from_dict(sequence_dict).T

In [9]:
df = df.apply(pd.to_numeric, errors='ignore')

/home/michael/anaconda3/envs/bio/lib/python3.8/site-packages/Bio/Seq.py:175: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  warnings.warn(


In [10]:
df.dtypes

name            object
length           int64
sequence        object
description     object
type            object
                 ...  
LPQYVYK-27-0     int64
KSHHIFT-43-3     int64
RSSHFAE-7-0      int64
LITAGDR-37-0     int64
LYCHTHK-10-0     int64
Length: 2796, dtype: object

In [15]:
print(sorted(df.columns))

['-0-0', 'A-1-0', 'AACLVHS-13-0', 'AC-2-0', 'ACISLFV-18-0', 'ACISLFV-18-1', 'ACK-3-0', 'ACLLTVL-13-0', 'ACLVHSR-12-0', 'ADTILLN-38-0', 'ADTILLN-56-0', 'AFS-3-0', 'AGCC-4-0', 'AGDRTRV-34-0', 'AGDRTRV-4444-0', 'AGSRTRR-12-0', 'AHAY-4-0', 'AHGTLRM-13-0', 'AHLLSM-6-0', 'AIFVLVT-19-0', 'AIFVLVT-19-1', 'AIIVNHI-53-0', 'AINSI-5-0', 'AIYN-4-0', 'AKRLCYV-23-0', 'AKRLCYV-23-1', 'ALCVHWR-19-0', 'ALG-3-0', 'ALL-3-0', 'ALSLPRV-8-0', 'AQAD-4-0', 'AQADEYE-83-0', 'AQADEYE-83-1', 'AQADEYE-83-2', 'AQH-3-0', 'ARHL-4-0', 'ASAQRSQ-287-16', 'ASAQRSQ-290-0', 'ASAQRSQ-290-1', 'ASAQRSQ-290-10', 'ASAQRSQ-290-11', 'ASAQRSQ-290-12', 'ASAQRSQ-290-13', 'ASAQRSQ-290-14', 'ASAQRSQ-290-15', 'ASAQRSQ-290-2', 'ASAQRSQ-290-3', 'ASAQRSQ-290-4', 'ASAQRSQ-290-5', 'ASAQRSQ-290-6', 'ASAQRSQ-290-7', 'ASAQRSQ-290-8', 'ASAQRSQ-290-9', 'ASTILL-6-0', 'ATSLALI-59-0', 'ATSLALI-59-1', 'AVLTQLR-20-0', 'AVLTQLR-20-1', 'AVQTQLR-20-0', 'AVTRLHA-7-0', 'AVYKMAM-34-0', 'AVYKMAM-34-1', 'AW-2-0', 'AWYIYLC-7-0', 'AXXLVHS-13-0', 'AY-2-0', 'AYFA

In [12]:
for col in df.columns: 
    print(col, df[col].sum()) 

name NC_045512MT350236MT350237MT350238MT350239MT350240MT350241MT350242MT350243MT350244MT350245MT350246MT350247MT350248MT350249MT350250MT350251MT350252MT350253MT350254MT350255MT350256MT350257MT350263MT350264MT350265MT350266MT350267MT350268MT350269MT350270MT350271MT350272MT350273MT350274MT350275MT350276MT350277MT350278MT350279MT350280MT350282MT344944MT344945MT344946MT344947MT344948MT344949MT344950MT344951MT344952MT344953MT344954MT344955MT344956MT344957MT344958MT344959MT344960MT344961MT344962MT344963MT345798MT345799MT345801MT345802MT345803MT345805MT345806MT345808MT345809MT345811MT345812MT345814MT345815MT345816MT345817MT345819MT345821MT345822MT345823MT345824MT345825MT345826MT345827MT345828MT345829MT345830MT345832MT345833MT345834MT345835MT345836MT345838MT345840MT345841MT345842MT345843MT345844MT345846MT345847MT345848MT345849MT345850MT345851MT345852MT345853MT345854MT345855MT345856MT345857MT345858MT345859MT345860MT345861MT345862MT345865MT345866MT345867MT345868MT345869MT345870MT345871MT345872MT

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 430
SNQLSIS-12-0 2
QSSKFQR-13-0 1
LLRRMTK-16-1 44
SRVLNGL-31-1 8
THSRRS-6-0 165
GCRSCYL-13-0 5
TVHLILS-13-0 1
GLYLPR-6-0 20
SVCNIRE-30-1 161
RFSRKLE-8-0 430
G-1-0 1455
LLMDHLK-13-0 165
IINN-4-0 1
NNLLVKL-39-1 1
NWLWMNS-12-0 2
N-1-0 1684
TKFKICV-14-0 1
WCCPTIT-7-0 165
TCILLCL-45-0 230
CF-2-0 165
SYIFP-5-0 430
MGR-3-0 429
LLFVLFS-24-1 228
QKKKKKK-15-2 1
IIKLMVT-17-0 231
IHQKITL-87-2 154
KQMCQFQ-18-0 427
GCDRKVR-59-0 1
MSGVWLH-82-1 89
QCN-3-0 430
NVRCKVH-24-1 1
CTIVFKR-2701-23 1
QKKKKKK-8-0 12
RHQIYIC-20-0 1
RSCCFRF-433-8 1
XIPSQVT-9-0 1
LQLQKLN-13-0 164
TNMKFLV-123-3 79
YIY-3-0 430
NVW-3-0 1
QRYY-4-0 231
KLLAIVV-19-0 165
DGEPCPW-46-1 3
ITHSVHR-9-0 231
EXXXXXX-11-1 1
LCLFLLH-24-1 429
FQLDTTS-9-0 230
EXXKKKK-16-0 1
LIINYCR-7-0 1
QKVSAFP-14-0 429
QFTHTFC-22-1 1
IPSSSRH-8-0 1
LSTSALL-17-0 231
LC-2-0 860
RTNGKT-6-0 1
TGLRCKC-17-1 164
CTIVFKR-2701-10 1
CLMLVIV-35-0 1
TNFKICV-20-0 1
LLRRMTK-21-1 1
QTIQLSI-13-0 1
RNVSKVA-7-0 430
PKGKMES-4409-32 1
LLSL-4-0 165
QKKKKKK-11-0 36
TYIALDL-29-0 1
HK-2

QR-2-0 429
LNGSCL-6-0 231
THADHTK-46-0 43
SYSQFCN-10-0 165
LNSSLNS-27-0 2
LLRISYG-7-0 1
TCISLFV-18-0 1
LLRXXXX-13-0 1
CLS-3-0 430
LHSTSRC-8-0 428
PKGKMES-4409-9 1
STPRTHA-10-0 430
AVLTQLR-20-0 135
HA-2-0 165
CYPHVIL-8-0 1
KXDTP-5-0 1
LEKTTEL-1293-21 1
SFYD-4-0 1
RSYNNSG-9-0 430
LLRRKKK-21-0 1
YSLS-4-0 2
CNCNM-5-0 165
LPT-3-0 165
LEKTTEL-1293-24 1
SIQCNTS-30-1 426
ET-2-0 430
HSLS-4-0 428
AIFVLVT-19-1 164
CVKLILV-30-2 1
CVVRSXX-9-1 1
LLRRMTK-20-0 1
NILCFT-6-0 430
SSNGTNL-7-0 165
PR-2-0 5
KVLLCLC-7-0 430
RSIINN-6-0 1
FT-2-0 595
SIQCNTS-30-2 3
LQFWCNF-11-0 430
LKLHLYS-15-2 3
FQLYSVN-32-0 165
PKGKMES-4409-37 1
LVSLSVV-8-0 231
LNFT-4-0 231
LLRRMTX-16-3 1
QTNQLSI-13-0 297
LXTFDLL-7-0 1
SCCY-4-0 430
CKQI-4-0 165
TILVLLL-16-0 165
PSKRDGN-24-0 1
FHLNCSS-25-0 1
RNNELLV-23-3 1
FTECCSL-17-1 1
LRDTLSL-34-2 1
WLSLGCM-15-0 3
CVKLILV-32-8 1
VQPVLHR-27-0 230
IIKISMV-26-0 231
TRWNLIR-14-0 165
KNYSILM-95-0 1
WTPKSAK-22-1 1
QDTSNSS-35-2 246
LPLILYY-16-0 165
NLITHMK-42-0 1
TDKYYNL-30-2 1
KVC-3-0 29
CTIVFKR-

In [13]:
df["-0-0"].sum()

41772

In [16]:
df.head()

,name,length,sequence,description,type,species,country,amino_list,-0-0,TED-3-0,...,KEKA-4-0,LEKTTEL-1293-15,IKGYCCY-15-1,HDDLSWL-7-0,FC-2-0,LPQYVYK-27-0,KSHHIFT-43-3,RSSHFAE-7-0,LITAGDR-37-0,LYCHTHK-10-0
NC_045512,NC_045512,29903,"(A, T, T, A, A, A, G, G, T, T, T, A, T, A, C, ...",Severe acute respiratory syndrome coronavirus ...,refseq,complete,Severe acute respiratory syndrome-related coro...,"[IKGLYLPR, QTNQLSISCRSVL, TNFKICVAVTRLHA, CTHA...",69,0,...,1,0,0,0,0,0,0,0,0,1
MT350236,MT350236,29865,"(A, C, T, T, T, C, G, A, T, C, T, C, T, T, G, ...",Severe acute respiratory syndrome coronavirus ...,complete,Homo sapiens,USA,"[TFDLL, ICSLNEL, NLCGCHSAACLVHSRSIINN, LLSLTGH...",54,1,...,0,0,0,1,1,1,0,0,0,0
MT350237,MT350237,29866,"(A, A, C, T, T, T, C, G, A, T, C, T, C, T, T, ...",Severe acute respiratory syndrome coronavirus ...,complete,Homo sapiens,USA,"[NFRSLVDLFSKRTLKSVWLSLGCMLSALTQYN, , LITVVDRTR...",14,0,...,0,0,0,0,0,0,0,0,0,0
MT350238,MT350238,29826,"(C, C, A, A, C, T, T, T, C, G, A, T, C, T, C, ...",Severe acute respiratory syndrome coronavirus ...,complete,Homo sapiens,USA,"[PTFDLL, ICSLNEL, NLCGCHSAACLVHSRSIINN, LLSLTG...",55,1,...,0,0,0,0,1,1,0,0,0,0
MT350239,MT350239,29897,"(T, G, T, T, T, A, T, A, C, C, T, T, C, C, C, ...",Severe acute respiratory syndrome coronavirus ...,complete,Homo sapiens,USA,"[CLYLPRXQTNQLSISCRSVL, TNFKICVAVTRLHA, CTHAV, ...",69,0,...,1,0,0,0,0,0,0,0,0,1


In [18]:
df['amino_list']

NC_045512    [IKGLYLPR, QTNQLSISCRSVL, TNFKICVAVTRLHA, CTHA...
MT350236     [TFDLL, ICSLNEL, NLCGCHSAACLVHSRSIINN, LLSLTGH...
MT350237     [NFRSLVDLFSKRTLKSVWLSLGCMLSALTQYN, , LITVVDRTR...
MT350238     [PTFDLL, ICSLNEL, NLCGCHSAACLVHSRSIINN, LLSLTG...
MT350239     [CLYLPRXQTNQLSISCRSVL, TNFKICVAVTRLHA, CTHAV, ...
                                   ...                        
MN988713     [IKGLYLPR, QTNQLSISCRSVL, TNFKICVAVTRLHA, CTHA...
MN938384     [QPTFDLL, ICSLNEL, NLCGCHSAACLVHSRSIINN, LLSLT...
MN975262     [IKGLYLPR, QTNQLSISCRSVL, TNFKICVAVTRLHA, CTHA...
MN985325     [IKGLYLPR, QTNQLSISCRSVL, TNFKICVAVTRLHA, CTHA...
MN908947     [IKGLYLPR, QTNQLSISCRSVL, TNFKICVAVTRLHA, CTHA...
Name: amino_list, Length: 826, dtype: object